In [1]:
import requests
from requests import get

import urllib.request
from urllib.request import urlopen
from urllib.request import urlretrieve

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time
import os
import pandas as pd
from multiprocessing import Pool

from io import BytesIO
from PIL import Image

In [3]:
url = 'https://kdata.or.kr/datavoucher/is/selectPortalSearch.do'

options = webdriver.ChromeOptions() 
# options.add_argument('headless') 
options.add_argument("window-size=1920x1080") 
options.add_argument('--no-sandbox')
options.add_argument("--disable-dev-shm-usage")
options.add_argument("disable-gpu") 
options.add_argument("--log-level=3")
options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(options=options)

driver.get(url)
driver.implicitly_wait(10)

driver.find_element(By.XPATH, '//*[@id="P01017006"]').send_keys(Keys.ENTER)
time.sleep(0.5)

driver.find_element(By.XPATH, '//*[@id="frm"]/div[2]/div/div/div/div[5]/a').send_keys(Keys.ENTER)
time.sleep(0.5)

company_name = []
explanation_company = []
company_phone = []
company_location = []
company_person = []
company_email = []


for i in range(1, 12):
    for p in range(2, 12): 
        detail_aTags = driver.find_elements(By.CSS_SELECTOR, 'div#listType div.product_list div.contents a')
        
        for idx in range(len(detail_aTags)):
            detail_aTags = driver.find_elements(By.CSS_SELECTOR, 'div#listType div.product_list div.contents a')
           
            detail_aTags[idx].send_keys(Keys.ENTER)
            driver.implicitly_wait(10)
            time.sleep(1)
               
            company_name.append(driver.find_element(By.CSS_SELECTOR, '#frm > div.sub_container > div > div > div > div.productSell_detail > div.productLeft_box > ul.left_item_01 > li:nth-child(1) > dl > dd').text)
            explanation_company.append(driver.find_element(By.CSS_SELECTOR, '#frm > div.sub_container > div > div > div > div.productSell_detail > div.productLeft_box > ul.left_item_01 > li:nth-child(2) > dl > dd').text)
            company_phone.append(driver.find_element(By.CSS_SELECTOR, '#frm > div.sub_container > div > div > div > div.productSell_detail > div.productLeft_box > ul.left_item_01 > li:nth-child(3) > dl > dd').text)
            company_location.append(driver.find_element(By.CSS_SELECTOR, '#frm > div.sub_container > div > div > div > div.productSell_detail > div.productLeft_box > ul.left_item_01 > li:nth-child(4) > dl > dd').text)
            company_person.append(driver.find_element(By.CSS_SELECTOR, '#frm > div.sub_container > div > div > div > div.productSell_detail > div.productLeft_box > ul.left_item_02 > li:nth-child(1) > dl > dd').text)
            company_email.append(driver.find_element(By.CSS_SELECTOR, '#frm > div.sub_container > div > div > div > div.productSell_detail > div.productLeft_box > ul.left_item_02 > li:nth-child(2) > dl > dd').text)
            
            driver.back()
            driver.implicitly_wait(10)            
            time.sleep(1)
            
        page_bar = driver.find_element(By.CSS_SELECTOR, 'div.pageing')
        pages = page_bar.find_elements(By.CSS_SELECTOR, 'a')
        
        pages[p].send_keys(Keys.ENTER)
        driver.implicitly_wait(10)
        time.sleep(1)

IndexError: list index out of range

In [5]:
import pandas as pd
df = pd.DataFrame()
df['기업명(영문)'] = company_name
df['설립일'] = explanation_company 
df['대표전화'] = company_phone
df['사업장소재지'] = company_location
df['담당자연락처'] = company_person
df['이메일'] = company_email

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   기업명(영문)  645 non-null    object
 1   설립일      645 non-null    object
 2   대표전화     645 non-null    object
 3   사업장소재지   645 non-null    object
 4   담당자연락처   645 non-null    object
 5   이메일      645 non-null    object
dtypes: object(6)
memory usage: 30.4+ KB


In [7]:
df.to_excel('./datavhoucher_crawling_20230726.xlsx', index = False)